<a href="https://colab.research.google.com/github/yongchanzzz/enzymology/blob/main/Fit_IC50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IC50 Curve Fitting Notebook for Google Colab
This notebook fits a 4-parameter logistic model to dose-response data.

Output files:
- Fit summary in TXT
- Plot in SVG

In [ ]:
#@title ### Cell 1: Install Required Packages
!pip install lmfit


In [ ]:
#@title ### Cell 2: Upload a CSV File
from google.colab import files
import io
import pandas as pd
uploaded = files.upload()  # Upload your CSV file
filename = list(uploaded.keys())[0]
data_all = pd.read_csv(io.BytesIO(uploaded[filename]))


In [ ]:

#@title ### Cell 3: User Parameters
# @markdown **User Inputs**
unit_in_M = 1e-9  #@param {type:"number", description:"Conversion factor from nM to M"}
inhibitor_column = "inhibitor"  #@param {type:"string", description:"Name of the inhibitor concentration column"}
velocity_column  = "velocity"   #@param {type:"string", description:"Name of the observed rate/velocity column"}

#@title ### Cell 4: Data Preprocessing
import numpy as np
# Convert inhibitor concentrations from nM to M
data_all[inhibitor_column] = data_all[inhibitor_column] * unit_in_M
# Separate blank (zero inhibitor) and treatment data
blank_data = data_all[data_all[inhibitor_column] <= 0]
data = data_all[data_all[inhibitor_column] > 0].copy()
# Log-transform inhibitor for fitting
data['log_inhibitor'] = np.log10(data[inhibitor_column])


In [ ]:
#@title ### Cell 4: Define Model and Fit
import lmfit
import datetime
import os, time
import numpy as np
from lmfit import Model
from google.colab import files

# 4‑parameter logistic function
def logistic_4p(logI, Bottom, Top, logIC50, HillSlope):
    return Bottom + (Top - Bottom) / (1 + 10**((logIC50 - logI) * HillSlope))

# build model
model = Model(logistic_4p, independent_vars=['logI'])
params = model.make_params(
    Bottom   = data[velocity_column].min(),
    Top      = data[velocity_column].max(),
    logIC50  = data['log_inhibitor'].mean(),
    HillSlope= 1
)
params['HillSlope'].set(min=0.1)

# fit
result = model.fit(data[velocity_column], params, logI=data['log_inhibitor'])

# prepare output
os.environ['TZ'] = 'Asia/Tokyo'
time.tzset()
timestamp    = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
summary_file = f"FitIC50_{timestamp}_summary.txt"

# ---  Build output lines ---
lines = []
lines.append("IC50 Curve Fit Results (GraphPad 4P Logistic Model)")
lines.append(f"Date and time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
lines.append(f"Input file: {filename}")
# Echo the equation
lines.append("Equation fitted: Y = Bottom + (Top – Bottom) / (1 + 10^((logIC50 – X) * HillSlope))")
lines.append("")  # blank line
lines.append("Calculated parameters:")

# --- each parameter + CI ---
for name, p in result.params.items():
    est     = p.value
    stderr  = p.stderr or 0
    ci_low  = est - 1.96*stderr
    ci_high = est + 1.96*stderr
    lines.append(f"{name:10} = {est:.5g} (95% CI: {ci_low:.5g} – {ci_high:.5g})")

# --- compute IC50 & its CI ---
logIC50 = result.params['logIC50']
ic50    = 10**logIC50.value
ci_lo   = logIC50.value - 1.96*logIC50.stderr
ci_hi   = logIC50.value + 1.96*logIC50.stderr
lines.append(f"IC50       = {ic50:.5g} (95% CI: {10**ci_lo:.5g} – {10**ci_hi:.5g})")

# --- Goodness-of-fit metrics (GraphPad Prism style) ---

# SSR and sy.x
residuals = result.residual
ss_res    = np.sum(residuals**2)
n_points  = len(data)
p_params  = len(result.params)
dof       = n_points - p_params
syx       = np.sqrt(ss_res / dof)

# R²
ss_tot    = np.sum((data[velocity_column] - data[velocity_column].mean())**2)
r2        = 1 - ss_res / ss_tot

# AICc calculation
# AICc = N*ln(SSR/N) + 2p + (2p(p+1)/(N-p-1))
aicc = (
    n_points * np.log(ss_res / n_points)
  + 2 * p_params
  + 2 * p_params * (p_params + 1) / (n_points - p_params - 1)
)

lines.append("")
lines.append("Goodness-of-fit metrics:")
lines.append(f"SSR (SSE)  = {ss_res:.5g}")
lines.append(f"sy.x       = {syx:.5g}")
lines.append(f"R²         = {r2:.5f}")
lines.append(f"AICc       = {aicc:.5f}")

# --- Replicate counts per concentration ---
from collections import defaultdict

rep_counts = data_all.groupby(inhibitor_column).size()
count_map  = defaultdict(list)
for conc, cnt in rep_counts.items():
    count_map[cnt].append(conc)

lines.append("")
lines.append("Number of replicates:")
for cnt in sorted(count_map, reverse=True):
    # format concentrations in scientific notation
    concs = ", ".join(f"{c:.3g}" for c in sorted(count_map[cnt]))
    lines.append(f"N={cnt}: {concs}")

# --- Session Info ---
import sys, platform, locale
from datetime import datetime as _dt

lines.append("")
lines.append("Session Info:")
lines.append(f"Python version {platform.python_version()} ({sys.version.split()[0]})")
lines.append(f"lmfit version {lmfit.__version__}")
import numpy, pandas, matplotlib
lines.append(f"NumPy version: {numpy.__version__}")
lines.append(f"pandas  version: {pandas.__version__}")
lines.append(f"Matplotlib version: {matplotlib.__version__}")
lines.append(f"Platform: {platform.platform()}")
now = datetime.datetime.now().astimezone()
lines.append(f"Time zone: {now.tzinfo} (UTC{now.utcoffset()})")


# write to file
with open(summary_file, 'w') as f:
    for L in lines:
        f.write(L + "\n")

# also print to stdout
for L in lines:
    print(L)

In [ ]:
#@title ### Cell 5: Plot SVG
import matplotlib.pyplot as plt

# @markdown **Output Plot Size (cm)**
width_cm = 8        #@param {type:"number", description:"Width in cm"}
height_cm = 6        #@param {type:"number", description:"Height in cm"}
# @markdown **Font and Text Options**
axis_tick_fontsize = 8      #@param {type:"number", description:"Font size for axis tick labels"}
axis_title_fontsize = 9     #@param {type:"number", description:"Font size for axis titles"}
# @markdown **Styling Options**
point_size = 5      #@param {type:"number", description:"Data point size"}
point_color = "black"       #@param {type:"string", description:"Data point color"}
blank_point_color = "green" #@param {type:"string", description:"Blank point color"}
curve_color = "black"       #@param {type:"string", description:"Fit curve color"}
curve_thickness = 1           #@param {type:"number", description:"Fit curve line thickness"}
axis_thickness = 1          #@param {type:"number", description:"Axis line/tick thickness"}
show_minor_ticks = False      #@param {type:"boolean", description:"Turn minor ticks on/off"}

# Prepare plot data
min_x = data[inhibitor_column].min()
pseudo_blank_x = min_x / 10
plot_norm = data.copy()
plot_bl = blank_data.copy(); plot_bl[inhibitor_column] = pseudo_blank_x

log_vals = data['log_inhibitor']
min_exp = int(np.floor(log_vals.min()))
max_exp = int(np.ceil(log_vals.max()))
exponents = np.arange(min_exp, max_exp + 1)
ticks = np.concatenate(([pseudo_blank_x], 10.0**exponents))
labels = ['Blank'] + [str(exp) for exp in exponents]

fig, ax = plt.subplots(figsize=(width_cm/2.54, height_cm/2.54))
fig.patch.set_facecolor('none')
ax.set_facecolor('none')

ax.scatter(plot_norm[inhibitor_column], plot_norm[velocity_column], s=point_size, color=point_color)
ax.scatter(plot_bl[inhibitor_column], plot_bl[velocity_column], s=point_size, color=blank_point_color)

x_seq = np.linspace(data['log_inhibitor'].min(), data['log_inhibitor'].max(), 300)
y_fit = logistic_4p(x_seq, result.params['Bottom'].value, result.params['Top'].value, result.params['logIC50'].value, result.params['HillSlope'].value)
ax.plot(10**x_seq, y_fit, color=curve_color, linewidth=curve_thickness)

ax.set_xscale('log')
ax.set_xticks(ticks)
ax.set_xticklabels(labels)
if show_minor_ticks:
    ax.xaxis.minorticks_on()
    ax.tick_params(axis='x', which='minor', width=axis_thickness)
else:
    ax.xaxis.minorticks_off()
ax.tick_params(width=axis_thickness, labelsize=axis_tick_fontsize)

for spine in ['bottom','left']:
    ax.spines[spine].set_linewidth(axis_thickness)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Log [Inhibitor] (M)', fontsize=axis_title_fontsize)
ax.set_ylabel('Velocity', fontsize=axis_title_fontsize)

plt.tight_layout()
plot_file = f"FitIC50_{timestamp}_plot.svg"
plt.savefig(plot_file, format='svg', transparent=True)
plt.show()


In [ ]:
#@title ### Cell 6: Download Results
files.download(summary_file)
files.download(plot_file)